In [15]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split 
from sklearn.datasets import make_blobs 
from sklearn import svm

from h import *
from W_unitary import *
from U_unitary import *
from qiskit.algorithms.optimizers import COBYLA, ADAM, SPSA, SLSQP, POWELL, L_BFGS_B, TNC, AQGD

%matplotlib inline
#plt.rcParams['figure.figsize'] = (10,6)
#plt.rcParams['figure.dpi'] = 100
#sns.set()

#%matplotlib inline
#%load_ext autoreload
#%autoreload 2

In [28]:
def MSE_loss(theta, X_train, y_train):
    
    # new_y_train = []
    # for i in range(0, len(y_train)):
    #     if y_train[i] == 0:
    #         new_y_train.append(-1)
    #     else:
    #         new_y_train.append(1)
    
    predictions = []
    for i in range(0, len(y_train)):
        predictions.append(h(X_train[i], theta, 2, 2, shots))
    
    error = []
    for i in range(0, len(predictions)):
        parity = predictions[i] - y_train[i]
        error.append(parity)

    norm = np.linalg.norm(error)
    return norm


def cross_validation_split(X,y, ele_per_split, i):
    X_train =  np.concatenate((X[:ele_per_split*i,:], X[ ele_per_split*(i+1) :,:]))
    X_test = X[ele_per_split*i:ele_per_split*(i+1),:]
    y_train = np.concatenate((y[ :ele_per_split*i], y[ele_per_split*(i+1):]))
    y_test = y[ ele_per_split*i:ele_per_split*(i+1)]
    return X_train, X_test, 2*y_train-1, 2*y_test-1

In [29]:


RANDOM_STATE=42
seed = np.random.seed(RANDOM_STATE)
theta = 2*np.pi*np.random.random(2*2*3)

n_samples = 1000
n_splits = 4

X, y = make_moons(n_samples, noise=0.8)
if n_samples%n_splits != 0:
    print("Preffered to have n_splits is divisor of n_samples")
ele_per_split = n_samples//n_splits

shots = 1024

In [50]:
thetas = np.zeros((len(theta), n_splits))
for i in range(n_splits):
    X_train, X_test, y_train, y_test = cross_validation_split(X,y, ele_per_split, i)

    objective_function = lambda theta: MSE_loss(theta, X_train, y_train)

    optimizer = COBYLA(maxiter=100)
    theta_opt = optimizer.minimize(objective_function, theta).x

    thetas[:,i] = theta_opt

    y_optimizer_train = np.zeros(len(y_train))
    for j in range(0, len(y_train)):
        y_optimizer_train[j] = (h(X_train[j], theta_opt, 2, 2, shots))

    y_optimizer_test = np.zeros(len(y_test))
    for j in range(0, len(y_test)):
        y_optimizer_test[j] = (h(X_test[j], theta_opt, 2, 2, shots))

    print("Split ", i)
    print("Optimal Theta ", theta_opt)
    print("Performance on training set ",1/2*np.sum(abs(y_optimizer_train-y_train)))
    print("Performance on test set ",1/2*np.sum(abs(y_optimizer_test-y_test)))

Split  0
Optimal Theta  [ 2.35341816  5.70465651  4.27370968  3.34986581  0.5246085   1.92306074
 -0.02954913  5.17674921  3.83403014  4.04531082  0.23293037  5.75266979]
Performance on training set  31.0
Performance on test set  13.0
Split  1
Optimal Theta  [ 3.08039691  8.12317012  5.9058596   3.74557053  0.98816011  1.33543614
  0.01522891  5.4405804   3.16047969  5.7933846  -1.2018166   6.09872312]
Performance on training set  29.0
Performance on test set  10.0
Split  2
Optimal Theta  [ 2.94169566  5.73958502  4.76849041  3.52727273  1.23237702  0.98160404
  0.20794524  5.36238518  3.62170162  5.52807217 -0.11865632  7.17376478]
Performance on training set  32.0
Performance on test set  13.0
Split  3
Optimal Theta  [3.35330497 5.65560505 4.24709261 4.1971825  0.80291862 1.08654605
 0.32246785 5.39986299 4.06026009 4.40646897 1.26045292 6.04977948]
Performance on training set  32.0
Performance on test set  14.0


In [51]:
theta_opt_avr = np.mean(thetas, axis=1)
X_val, y_val = make_moons(n_samples//n_splits, noise=0.8)

y_optimizer_val = np.zeros(len(y_val))
for j in range(0, len(y_val)):
    y_optimizer_val[j] = (h(X_val[j], theta_opt, 2, 2, shots))

print("Average Theta ", theta_opt_avr)
print("Performance on training set ",1/2*np.sum(abs(y_optimizer_train-y_train)))
print("Performance on test set ",1/2*np.sum(abs(y_optimizer_test-y_test)))

Average Theta  [2.93220393 6.30575418 4.79878807 3.70497289 0.88701606 1.33166174
 0.12902322 5.34489444 3.66911789 4.94330914 0.04322759 6.26873429]
Performance on training set  32.0
Performance on test set  14.0
